<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/main/Hg_Edgar_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# 📦 Install required packages
!pip install geopandas contextily mapclassify

# ──────────────────────────────────────────────────────────────────────────────
# 🚀 Imports
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from google.colab import drive

# ──────────────────────────────────────────────────────────────────────────────
# 🔌 Mount Drive & Load Data
drive.mount('/content/drive')
df = pd.read_excel(
    '/content/drive/My Drive/ColabDataFolders/Hg-EDGAR/Hg_D_m_2000_2022.xlsx',
    header=0
)

# ──────────────────────────────────────────────────────────────────────────────
# 📊 Preprocess: annual total
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
df[months] = df[months].apply(pd.to_numeric, errors='coerce')
df['annual_total'] = df[months].sum(axis=1)

# ──────────────────────────────────────────────────────────────────────────────
# 🔢 Country stats: total & percent change
country_year = df.groupby(['Country_code_A3','Name','Year'])['annual_total'].sum().reset_index()
pivot = country_year.pivot_table(index=['Country_code_A3','Name'], columns='Year', values='annual_total', fill_value=0)
pivot = pivot[(pivot.get(2000,0)>0)&(pivot.get(2022,0)>0)].reset_index()
pivot['Absolute_change'] = pivot[2022] - pivot[2000]
pivot['Percent_change'] = pivot['Absolute_change']/pivot[2000]*100

country_totals = pivot.rename(columns={2000:'2000_value',2022:'2022_value'})
country_aggregates = df.groupby(['Country_code_A3','Name'])['annual_total'].sum().reset_index()
country_aggregates['total_hg_all_years'] = country_aggregates['annual_total']
world_total = country_aggregates['annual_total'].sum()
country_aggregates['fraction_of_global'] = country_aggregates['annual_total']/world_total*100

# ──────────────────────────────────────────────────────────────────────────────
# 🌍 Load world geometries & merge

# Load world geometries (ISO‑A3 codes are in the `id` field)
world = gpd.read_file(
    "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"
)
map_total = world.merge(
    country_aggregates,
    left_on='id',
    right_on='Country_code_A3',
    how='left'
)

map_pct = world.merge(
    country_totals[['Country_code_A3','Percent_change']],
    left_on='id',
    right_on='Country_code_A3',
    how='left'
)



# ──────────────────────────────────────────────────────────────────────────────
# 🎨 Plot function
def plot_choropleth(gdf, column, title, fname):
    fig, ax = plt.subplots(figsize=(16,8))
    gdf.plot(
        column=column,
        ax=ax,
        legend=True,
        scheme='quantiles',
        cmap='viridis',
        missing_kwds={'color':'lightgrey','label':'No data'}
    )
    ax.set_title(title, fontsize=20)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(f'visualizations/{fname}', dpi=300)
    plt.close()

# ──────────────────────────────────────────────────────────────────────────────
# 🖼️ Ensure output folder exists
import os
os.makedirs('visualizations', exist_ok=True)

# ──────────────────────────────────────────────────────────────────────────────
# 📈 Generate maps
plot_choropleth(map_total, 'total_hg_all_years', 'Total Hg Emissions (2000–2022)', 'map_total_emissions.png')
plot_choropleth(map_pct,   'Percent_change', '% Change Hg Emissions (2000→2022)', 'map_percent_change.png')

print("✅ Completed — check /content/visualizations for your two PNGs.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Completed — check /content/visualizations for your two PNGs.


In [ ]:
# 📦 Install required packages
!pip install pandas

# ────────────────────────────────────────────────────────────────────────────────
# 🚀 Imports
import pandas as pd
from google.colab import drive
import os

# ────────────────────────────────────────────────────────────────────────────────
# 🔌 Mount Drive & Load Data
drive.mount('/content/drive')
df = pd.read_excel(
    '/content/drive/My Drive/ColabDataFolders/Hg-EDGAR/Hg_D_m_2000_2022.xlsx',
    header=0
)

# ────────────────────────────────────────────────────────────────────────────────
# 📊 Preprocess: annual total
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
df[months] = df[months].apply(pd.to_numeric, errors='coerce')
df['annual_total'] = df[months].sum(axis=1)

# ────────────────────────────────────────────────────────────────────────────────
# 🔍 Extract necessary information
# First, we need to extract the country names and emission sources
# Let's create a dataframe with country, source type, total value, and percent change

# Get country totals by year for percent change calculation
country_source_year = df.groupby(['Name', 'ipcc_code_2006_for_standard_report_name', 'Year'])['annual_total'].sum().reset_index()

# Create pivot table to calculate change between 2000 and 2022
pivot = country_source_year.pivot_table(
    index=['Name', 'ipcc_code_2006_for_standard_report_name'],
    columns='Year',
    values='annual_total',
    fill_value=0
).reset_index()

# Filter for entries that have data in both 2000 and 2022
valid_data = pivot[(pivot.get(2000, 0) > 0) & (pivot.get(2022, 0) > 0)].copy()

# Calculate total emissions (sum of all years)
country_source_total = df.groupby(['Name', 'ipcc_code_2006_for_standard_report_name'])['annual_total'].sum().reset_index()
country_source_total.rename(columns={'annual_total': 'total_emissions'}, inplace=True)

# Calculate absolute and percent change
valid_data['total_value'] = valid_data[2022]  # Using 2022 as the "total" value as requested
valid_data['absolute_change'] = valid_data[2022] - valid_data[2000]
valid_data['percent_change'] = (valid_data['absolute_change'] / valid_data[2000] * 100).round(2)

# Format the percent change as requested (e.g., "+14%")
valid_data['formatted_change'] = valid_data['percent_change'].apply(
    lambda x: f"+{x:.2f}%" if x >= 0 else f"{x:.2f}%"
)

# Create final dataframe for CSV export
export_data = valid_data[['Name', 'ipcc_code_2006_for_standard_report_name', 'total_value', 'formatted_change']]
export_data.columns = ['Country', 'Source', 'Total Value', 'Change']

# ────────────────────────────────────────────────────────────────────────────────
# 💾 Export to CSV
output_dir = 'emissions_data'
os.makedirs(output_dir, exist_ok=True)
csv_path = f'{output_dir}/mercury_emissions_by_country_source.csv'
export_data.to_csv(csv_path, index=False)

print(f"✅ Completed — CSV file saved to {csv_path}")

# Display the first few rows of the CSV data
print("\nPreview of the exported data:")
print(export_data.head())

# ────────────────────────────────────────────────────────────────────────────────
# 📊 Summary Statistics
print(f"\nTotal number of country-source combinations: {len(export_data)}")
print(f"Number of unique countries: {export_data['Country'].nunique()}")
print(f"Number of unique emission sources: {export_data['Source'].nunique()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Completed — CSV file saved to emissions_data/mercury_emissions_by_country_source.csv

Preview of the exported data:
       Country                                         Source   Total Value  \
0  Afghanistan                              Cement production  1.122244e-06   
1  Afghanistan                               Glass Production  1.663086e-08   
2  Afghanistan  Main Activity Electricity and Heat Production  1.560413e-05   
3  Afghanistan      Manufacturing Industries and Construction  3.432336e-05   
4  Afghanistan                                  Non-Specified  3.284566e-08   

      Change  
0    +16.65%  
1    +98.15%  
2  +2572.49%  
3  +9012.50%  
4   +551.39%  

Total number of country-source combinations: 1795
Number of unique countries: 213
Number of unique emission sources: 13


In [ ]:
# 📦 Install required packages
!pip install pandas

# ────────────────────────────────────────────────────────────────────────────────
# 🚀 Imports
import pandas as pd
from google.colab import drive
import os

# ────────────────────────────────────────────────────────────────────────────────
# 🔌 Mount Drive & Load Data
drive.mount('/content/drive')
df = pd.read_excel(
    '/content/drive/My Drive/ColabDataFolders/Hg-EDGAR/Hg_D_m_2000_2022.xlsx',
    header=0
)

# ────────────────────────────────────────────────────────────────────────────────
# 📊 Preprocess: annual total
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
df[months] = df[months].apply(pd.to_numeric, errors='coerce')
df['annual_total'] = df[months].sum(axis=1)

# ────────────────────────────────────────────────────────────────────────────────
# 🔍 Extract necessary information
# First, we need to extract the country names and emission sources
# Let's create a dataframe with country, source type, total value, and percent change

# Get country totals by year for percent change calculation
# Sum values for any duplicate combinations of country and source
country_source_year = df.groupby(['Name', 'ipcc_code_2006_for_standard_report_name', 'Year'])['annual_total'].sum().reset_index()

# Check for any duplicate country-source combinations and report them
duplicate_check = df.groupby(['Name', 'ipcc_code_2006_for_standard_report_name', 'Year']).size().reset_index(name='count')
duplicates = duplicate_check[duplicate_check['count'] > 1]
if not duplicates.empty:
    print(f"Note: Found {len(duplicates)} country-source-year combinations with multiple entries in the original data.")
    print("These have been SUMMED to get the total emissions.")

# Create pivot table to calculate change between 2000 and 2022
pivot = country_source_year.pivot_table(
    index=['Name', 'ipcc_code_2006_for_standard_report_name'],
    columns='Year',
    values='annual_total',
    aggfunc='sum',  # Explicitly use sum to handle any duplicates
    fill_value=0
).reset_index()

# Filter for entries that have data in both 2000 and 2022
valid_data = pivot[(pivot.get(2000, 0) > 0) & (pivot.get(2022, 0) > 0)].copy()

# Calculate total emissions (sum of all years)
country_source_total = df.groupby(['Name', 'ipcc_code_2006_for_standard_report_name'])['annual_total'].sum().reset_index()
country_source_total.rename(columns={'annual_total': 'total_emissions'}, inplace=True)

# Calculate absolute and percent change
valid_data['total_value'] = valid_data[2022]  # Using 2022 as the "total" value as requested
valid_data['absolute_change'] = valid_data[2022] - valid_data[2000]
valid_data['percent_change'] = (valid_data['absolute_change'] / valid_data[2000] * 100).round(2)

# Format the percent change as requested (e.g., "+14%")
valid_data['formatted_change'] = valid_data['percent_change'].apply(
    lambda x: f"+{x:.2f}%" if x >= 0 else f"{x:.2f}%"
)

# Create final dataframe for CSV export
export_data = valid_data[['Name', 'ipcc_code_2006_for_standard_report_name', 'total_value', 'formatted_change']]
export_data.columns = ['Country', 'Source', 'Total Value', 'Change']

# ────────────────────────────────────────────────────────────────────────────────
# 💾 Export to CSV
output_dir = 'emissions_data'
os.makedirs(output_dir, exist_ok=True)
csv_path = f'{output_dir}/mercury_emissions_by_country_source.csv'
export_data.to_csv(csv_path, index=False)

print(f"✅ Completed — CSV file saved to {csv_path}")

# Display the first few rows of the CSV data
print("\nPreview of the exported data:")
print(export_data.head())

# ────────────────────────────────────────────────────────────────────────────────
# 📊 Summary Statistics
print(f"\nTotal number of country-source combinations: {len(export_data)}")
print(f"Number of unique countries: {export_data['Country'].nunique()}")
print(f"Number of unique emission sources: {export_data['Source'].nunique()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Note: Found 13246 country-source-year combinations with multiple entries in the original data.
These have been SUMMED to get the total emissions.
✅ Completed — CSV file saved to emissions_data/mercury_emissions_by_country_source.csv

Preview of the exported data:
       Country                                         Source   Total Value  \
0  Afghanistan                              Cement production  1.122244e-06   
1  Afghanistan                               Glass Production  1.663086e-08   
2  Afghanistan  Main Activity Electricity and Heat Production  1.560413e-05   
3  Afghanistan      Manufacturing Industries and Construction  3.432336e-05   
4  Afghanistan                                  Non-Specified  3.284566e-08   

      Change  
0    +16.65%  
1    +98.15%  
2  +2572.49%  
3  +9012.50%  
4   +551.39%  

Total number of country-source combinatio